<a href="https://colab.research.google.com/github/dparcon/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")


In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4bed = spark.sql("""
    SELECT
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        YEAR(date)
    ORDER BY
        year
""")
avg_price_4bed.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3bed_3bath = spark.sql("""
    SELECT
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        YEAR(date)
    ORDER BY
        year
""")
avg_price_3bed_3bath.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
|2022|    292725.69|
+----+-------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_3bed_3bath_2ksqft = spark.sql("""
    SELECT
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY
        YEAR(date)
    ORDER BY
        year
""")
avg_price_3bed_3bath_2ksqft.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
|2022|    290242.99|
+----+-------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

average_price_view_rating = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
""")

end_time = time.time()
execution_time = end_time - start_time

average_price_view_rating.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  81|   1053472.79|
|  76|   1058802.78|
|  91|   1137372.73|
|  93|   1026006.06|
|  52|    733780.26|
|  86|   1070444.25|
|  94|    1033536.2|
|  57|     734340.5|
|  54|    798684.82|
|  96|   1017815.92|
|  92|    970402.55|
|  64|    767036.67|
|  61|    746877.59|
|  88|   1031719.35|
|  72|    780914.67|
|  59|     791453.0|
+----+-------------+
only showing top 20 rows

--- 1.292443037033081 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

cached_results = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
""")

end_time = time.time()
execution_time = end_time - start_time

cached_results.show()
print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  81|   1053472.79|
|  76|   1058802.78|
|  91|   1137372.73|
|  93|   1026006.06|
|  52|    733780.26|
|  86|   1070444.25|
|  94|    1033536.2|
|  57|     734340.5|
|  54|    798684.82|
|  96|   1017815.92|
|  92|    970402.55|
|  64|    767036.67|
|  61|    746877.59|
|  88|   1031719.35|
|  72|    780914.67|
|  59|     791453.0|
+----+-------------+
only showing top 20 rows

--- 0.6445093154907227 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/home_sales_partitioned.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

parquet_results = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS average_price
    FROM
        home_sales_parquet
    GROUP BY
        view
    HAVING
        AVG(price) >= 350000
""")

end_time = time.time()
execution_time = end_time - start_time

parquet_results.show()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")